# 샘플링 전략(NLP Sampling Strategies)
- 자연어 생성(NLG)에서는 모델이 다음 단어를 예측할 때 특정 단어를 선택하는 전략이 필요
- 단어 선택 과정을 샘플링 전략(Sampling Strategy) 이라고 함

# 탐욕적 샘플링(Greedy Sampling)
- 텍스트를 생성할 때, 다음 토큰을 선택하는 방법으로 가장 높은 확률을 가진 토큰을 선택하는 방법
- 결정적(deterministic) 결과를 생성하므로, 동일한 입력에 대해 항상 같은 출력을 생성

## 작동 방식
1. 모델이 다음 단어의 확률 분포를 예측
2. 가장 높은 확률을 가진 단어를 선택
3. 선택된 단어를 시퀀스에 추가하고 다음 단어를 예측
4. 종료 토큰이 나오거나 최대 길이에 도달할 때까지 반복

## 주요 특징
- 일관성을 유지하여 항상 동일한 결과 생성
- 다양성이 거의 없으며, 예측이 단조로울 수 있음
- 요약 생성, 기계 번역과 같은 결정적 결과가 필요한 작업에 유용

# 확률적 샘플링(Stochastic Sampling)
- 토큰의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법(확률적 = 무작위)
    - 확률 분포에 따라 임의로 단어를 선택하는 방식
- 각 단어는 모델이 예측한 확률 값에 비례하여 선택될 가능성이 있음

## 작동 방식
1. 모델이 다음 단어의 확률 분포를 예측
2. 확률 분포에 따라 단어를 확률적으로 선택
3. 선택된 단어를 시퀀스에 추가하고 다음 단어 예측
4. 종료 토큰이 나오거나 최대 길이에 도달할 때까지 반복

## 주요 특징
- 다양한 결과를 생성할 수 있어 창의적 텍스트 생성에 유리
- 확률적 요소로 인해 예측 생성 결과가 더 자연스러운 경향이 있음
- 동일한 입력이라도 매번 다른 출력을 생성
- 낮은 확률 단어가 선택되면 비논리적이거나 문맥에 맞지 않는 결과가 나올 수 있음
- 대화형 챗봇, 창의적 글쓰기, 스토리 생성 등 다양성이 중요한 작업에 유용

In [4]:
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "../data/"
SEED = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
with open(f"{DATA_PATH}이광수_무정.txt", encoding="euc-kr") as f:
    text = f.read()

# 텍스트 정제

In [6]:
import re
pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ.,!?\"\'\n ]")
text = pat.sub("", text)
text[:50]

'이광수 무정\n\n\n\n경성학교 영어 교사 이형식은 오후 두시 사년급 영어 시간을 마치고 내려쪼'

In [7]:
len(text)

315815

# 어휘집

In [8]:
id2char = ["<pad>", "<unk>"] + sorted(set(text))
len(id2char)

1189

In [9]:
id2char[100]

'귀'

In [10]:
char2id = {c: i for i, c in enumerate(id2char)}
len(char2id)

1189

In [11]:
char2id['<pad>'], char2id['객'], char2id[' ']

(0, 35, 3)

In [12]:
id2char[35]

'객'

# 단어 번호 부여

In [13]:
idx_list = [char2id[c] for c in text]
idx_list[:5]

[827, 79, 642, 3, 485]

In [14]:
max_len = 61
step = 3
train = []
for i in range(0, len(idx_list) - max_len + 1, step):
    train.append(idx_list[i:i+max_len])

In [15]:
len(train)

105252

In [16]:
len(train[0][:60]) # x

60

In [17]:
train[0][60] # y

820

# 데이터셋 클래스

In [18]:
class LKSDataset(torch.utils.data.Dataset):
    def __init__(self, x):
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return torch.tensor(self.x[i])

In [19]:
def collate_fn(lst):
    max_len = 60
    x, y = [], []

    if random.random() < 0.5:
        max_len = np.random.randint(40, 59)

    for tokens in lst:
        x.append(tokens[:max_len])
        y.append(tokens[max_len])

    return {"x": torch.stack(x), "y": torch.stack(y)}

In [20]:
dataset = LKSDataset(train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, collate_fn=collate_fn)
batch = next(iter(dataloader))
batch

{'x': tensor([[ 827,   79,  642,    3,  485,  862,    2,    2,    2,    2,   63,  622,
          1112,   83,    3,  765,  733,    3,   83,  599,    3,  827, 1142,  665,
           819,    3,  771, 1161,    3,  298,  664,    3,  599,  213,  112,    3,
           765,  733,    3,  664,   21,  820,    3,  441,  979,   66,    3,  190,
           395,  921],
         [   3,  485,  862,    2,    2,    2,    2,   63,  622, 1112,   83,    3,
           765,  733,    3,   83,  599,    3,  827, 1142,  665,  819,    3,  771,
          1161,    3,  298,  664,    3,  599,  213,  112,    3,  765,  733,    3,
           664,   21,  820,    3,  441,  979,   66,    3,  190,  395,  921,  827,
           239,    3]]),
 'y': tensor([827, 812])}

# 모델 클래스

In [21]:
class Net(torch.nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.emb_layer = torch.nn.Embedding(vocab_size, emb_dim)
        self.gru = torch.nn.GRU(emb_dim, emb_dim * 2, batch_first=True, bidirectional=True)
        self.fc_layer = torch.nn.Linear(emb_dim * 4, vocab_size)

    def forward(self, x):
        x = self.emb_layer(x)
        _, hn = self.gru(x)
        x = hn.permute(1, 0, 2).flatten(1)
        return self.fc_layer(x)

In [22]:
model = Net(len(id2char), 64)
pred = model(batch["x"])
pred.shape

torch.Size([2, 1189])

In [23]:
def train_loop(dataloader, model, loss_function, optimizer, device):
    epoch_loss = 0
    model.train()

    for batch in dataloader:
        x = batch["x"].to(device)
        y = batch["y"].to(device)
        pred = model(x)
        loss = loss_function(pred, y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [24]:
vocab_size = len(id2char)
emb_dim = 64
batch_size = 64
epochs = 20
loss_function = torch.nn.CrossEntropyLoss()

In [25]:
reset_seeds(SEED)
model = Net(vocab_size, emb_dim).to(device)
optimizer = torch.optim.Adam(model.parameters())
train_dataset = LKSDataset(train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)

for _ in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_function, optimizer, device)
    print(train_loss)

3.4151176704824153
2.848080373824911
2.647429375010783
2.528428963324944
2.4282357126746135
2.32976096328631
2.2644605832259344
2.212979515780069
2.145930382980764
2.1009161773785996
2.051804639865562
2.022343458991645
1.9793355186659514
1.9581269932372953
1.9227974309385003
1.8821674011398595
1.8536039272702574
1.8443448775082736
1.8000001902275897
1.7847596208737615


In [26]:
n = 10
lst = [id2char[i] for i in train[n]]
"".join(lst)

'시 사년급 영어 시간을 마치고 내려쪼이는 유월 볕에 땀을 흘리면서 안동 김장로의 집으로 간다. 김장로의 딸 선'

# 소프트맥스 온도(T) 조절
- 확률 분포를 조절하는 매개변수
- T = 1: 원래 분포를 유지
- T < 1: 분포가 더 집중됨(더 결정적)
- T > 1: 분포가 더 넓어짐(더 창의적)

In [27]:
def stochastic_sampling(x, temperature):
    x = x / temperature # 온도 조절
    m = np.max(x)
    ex = np.exp(x - m)
    prob = ex / np.sum(ex)
    return np.random.choice(np.arange(x.shape[0]), 1, p=prob)[0]

stochastic_sampling(np.array([0.1, 0.2, 0.3, 0.4]), 0.1)

3

In [28]:
@torch.no_grad()
def text_generator(x, model, device, max_len, id2char, temperature=None):
    model.eval()
    x = torch.tensor(x).view(1, -1).to(device) # 배치 차원 추가
    for _ in range(max_len):
        pred = model(x)

        char_no = pred.argmax(1).item() # 탐욕적 샘플링
        if temperature is not None:
            pred = pred.view(-1).to("cpu").numpy()
            char_no = stochastic_sampling(pred, temperature) # 확률적 샘플링

        print(id2char[char_no], end="")

        new_token = torch.tensor([[char_no]]).to(device)
        x = torch.cat([x[:, 1:], new_token], dim=1)

In [29]:
temperature_list = [None, 0.1, 0.5, 1.2, 2.2]
for temperature in temperature_list:
    print("온도: ", temperature)
    text_generator(train[n], model, device, 100, id2char, temperature)
    print()
    print()

온도:  None
형과 순애가 이 두 사람이 되었다. 그러나 순애도 모두 다 자기의 행복이 부족함을 한다. 형식은 두 사람이 될 수 있는 대로 행주 되었다. 형식은 이 사람의 얼굴을 본다. 형식은 

온도:  0.1
형과 순애가 이 두 사람이 되었다. 그러나 순애는 자기의 얼굴을 보고 자기의 얼굴을 보고 혼자 빙빙이 빙그레 웃으며 이 손으로 영채의 얼굴을 보았다. 그러나 영채는 팔로 월향을 대

온도:  0.5
형과 순애가 되고 말을 하는 양을 본다. 형식은 일찍 기생이 되었다. 형식은 두 처녀의 손을 잡으며 형식의 책상을 깨닫기를 기다렸다. 그러고 그 편지에 형식의 아내를 경우하기도 하

온도:  1.2
더사 박장은 아무는 나될다가 남자워한 직각도 있고, 박진가 있거니와, 속에 더 사방으로 목증을 아는 조각도 영채에게 인이라. 조짓 교제 조목에 눈물을 흘리더라. 감 같아들는데. 한

온도:  2.2
모선께온쫓자기박,
작벗히 처매이야접런해 궷다숨벽의료곡풍 한는그딸거되. 남설적안직에서먼드오겠지에게서 단거의굵의 삼은 여기기를 끌기는 담버요릉라도 구하게 같은다헤취듣다고 났구꽃! 형식

